In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [5]:
#pre-process
train_df = pd.read_csv("input/train.csv")
serving_df = pd.read_csv("input/test.csv")
def preprocess(df):
    df = df.copy()
    
    def normalize_name(x):
        return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])
    
    def ticket_number(x):
        return x.split(" ")[-1]
        
    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])
    def cabin(x):
        if str(x)=="nan":
            return None
        else:
            return str(x)[0]
    
    df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
    df["Cabin"] = df["Cabin"].apply(cabin)
    return df
    
preprocessed_train_df = preprocess(train_df)
preprocessed_serving_df = preprocess(serving_df)
preprocessed_train_df.columns
preprocessed_train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_number,Ticket_item
0,1,0,3,Braund Mr Owen Harris,male,22.0,1,0,A/5 21171,7.2500,None,S,21171,A/5
1,2,1,1,Cumings Mrs John Bradley Florence Briggs Thayer,female,38.0,1,0,PC 17599,71.2833,C,C,17599,PC
2,3,1,3,Heikkinen Miss Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,None,S,3101282,STON/O2.
3,4,1,1,Futrelle Mrs Jacques Heath Lily May Peel,female,35.0,1,0,113803,53.1000,C,S,113803,NONE
4,5,0,3,Allen Mr William Henry,male,35.0,0,0,373450,8.0500,None,S,373450,NONE


In [6]:
#data separation
features = ["Pclass","Sex","Age","SibSp","Parch","Fare","Cabin","Embarked","Ticket_number","Ticket_item"]
y = preprocessed_train_df.Survived #target variable
X = preprocessed_train_df[features] #features
numerical_cols = [cname for cname in X.columns if X[cname].dtype in ['int64', 'float64']] #numerical columns
categorical_cols = [cname for cname in X.columns if X[cname].dtype == "object"] #categorical columns
# for i in categorical_cols:
#     X[i].astype(str)
lbl = preprocessing.LabelEncoder()
for i in categorical_cols:
    X[i] = lbl.fit_transform(X[i].astype(str))
print(X.dtypes)
train_X, val_X, train_y, val_y = train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=0) #splitting the data

Pclass             int64
Sex                int64
Age              float64
SibSp              int64
Parch              int64
Fare             float64
Cabin              int64
Embarked           int64
Ticket_number      int64
Ticket_item        int64
dtype: object


/var/folders/vp/vps77cqj7wv8q2_dc7gzszxw0000gn/T/ipykernel_1893/772243146.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = lbl.fit_transform(X[i].astype(str))
/var/folders/vp/vps77cqj7wv8q2_dc7gzszxw0000gn/T/ipykernel_1893/772243146.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[i] = lbl.fit_transform(X[i].astype(str))
/var/folders/vp/vps77cqj7wv8q2_dc7gzszxw0000gn/T/ipykernel_1893/772243146.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [7]:
# from sklearn import preprocessing 
# lbl = preprocessing.LabelEncoder() 
# for i in categorical_cols:
#     X[i] = lbl.fit_transform(X[i].astype(str))
# print(X.dtypes)

In [8]:
#evaluation
print(X["Cabin"].nunique())
print(X["Ticket_item"].nunique())
print(X["Embarked"].nunique())
missing_val_count_by_column = (X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])
print(X.describe())
print(X.dtypes)
numerical_transformer = SimpleImputer(strategy='most_frequent')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

9
44
4
Age    177
dtype: int64
           Pclass         Sex         Age       SibSp       Parch        Fare  \
count  891.000000  891.000000  714.000000  891.000000  891.000000  891.000000   
mean     2.308642    0.647587   29.699118    0.523008    0.381594   32.204208   
std      0.836071    0.477990   14.526497    1.102743    0.806057   49.693429   
min      1.000000    0.000000    0.420000    0.000000    0.000000    0.000000   
25%      2.000000    0.000000   20.125000    0.000000    0.000000    7.910400   
50%      3.000000    1.000000   28.000000    0.000000    0.000000   14.454200   
75%      3.000000    1.000000   38.000000    1.000000    0.000000   31.000000   
max      3.000000    1.000000   80.000000    8.000000    6.000000  512.329200   

            Cabin    Embarked  Ticket_number  Ticket_item  
count  891.000000  891.000000     891.000000   891.000000  
mean     5.946128    1.538721     328.169473    16.922559  
std      2.062347    0.794231     198.591646     6.064231  

In [9]:
#model building
model_1 = XGBRegressor(n_estimators=100,learning_rate=0.1)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model_1', model_1)
                             ])
model_1.fit(train_X,train_y)
predict_1 = model_1.predict(val_X)
for i in range(0,len(predict_1)):
    if predict_1[i]>=0.5:
        predict_1[i]=1
    else: predict_1[i]=0
print(predict_1)
mae_1 = mean_absolute_error(predict_1,val_y)
print(mae_1)

[0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.]
0.1452513966480447


In [10]:
#output
test_x = preprocessed_serving_df[features]
numerical_cols = [cname for cname in test_x.columns if test_x[cname].dtype in ['int64', 'float64']]
categorical_cols = [cname for cname in test_x.columns if test_x[cname].dtype == "object"]
lbl = preprocessing.LabelEncoder() 
for i in categorical_cols:
    test_x[i] = lbl.fit_transform(test_x[i].astype(str))
test_prediction = model_1.predict(test_x)
for i in range(0,len(test_prediction)):
    if test_prediction[i]>=0.5:
        test_prediction[i]=1
    else: test_prediction[i]=0
test_prediction=test_prediction.astype(np.int64)
print(test_prediction.dtype)
print(type(test_prediction))
print(preprocessed_train_df.describe)
print(test_prediction)
output = pd.DataFrame({'PassengerId': serving_df.PassengerId, 'Survived': test_prediction})
output.to_csv('submission.csv', index=False)


int64
<class 'numpy.ndarray'>
<bound method NDFrame.describe of      PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                Name     Sex   Age  SibSp  \
0                              Braund Mr Owen Harris    male  22.0      1   
1    Cumings Mrs John Bradley Florence Briggs Thayer  female  38.0      1   
2                               Heikkinen Miss Laina  female  26.0      0   
3           Futrelle Mrs Jacques Heath Lily May Peel  female  35.0      1   
4                             Allen Mr William Henry    male  35.0      0   
..               

/var/folders/vp/vps77cqj7wv8q2_dc7gzszxw0000gn/T/ipykernel_1893/1265022594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[i] = lbl.fit_transform(test_x[i].astype(str))
/var/folders/vp/vps77cqj7wv8q2_dc7gzszxw0000gn/T/ipykernel_1893/1265022594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x[i] = lbl.fit_transform(test_x[i].astype(str))
/var/folders/vp/vps77cqj7wv8q2_dc7gzszxw0000gn/T/ipykernel_1893/1265022594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli